In [55]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
#список пользователей по leader-id  и email из ple
users=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/ple_user.csv', \
                              sep=';',encoding='utf-8')

#список zoom_id  мероприятий и их типов
events=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_event.csv', \
                              sep=';',encoding='utf-8')
#иногда после номера мероприятия встречается пароль - выпилим его
events.meeting_id=events.meeting_id.str.extract(r'\D*(\d*)\D*').astype('int64')

#список всех зум-сессий. В sql запросе групировка для вывода: lmp.user_name, lmp.meeting_id, lmp.email
sessions=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_sesion.csv', \
                              sep=';',encoding='utf-8')
sessions.meeting_id=sessions.meeting_id.astype('int64')


#Статистика вовлеченности за день
stat=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/engagement_v20_21_11_2020.csv', \
                              sep=';',encoding='utf-8')

In [62]:
# Сшиваем по полю meeting_id
sessions_merged = pd.merge(
    sessions,
    events,
    on=['meeting_id'],
    how='left' ).fillna(0)

# Сшиваем  c users по полю email
sessions_merged = pd.merge(
    sessions_merged,
    users,
    on=['email'],
    how='left' ).fillna(0)

sessions_merged.leaderID=sessions_merged.leaderID.astype('int')
sessions_merged['zoom_name_leaderID']= sessions_merged.user_name.str.extract(r'\D*(\d*)\D*')
sessions_merged['zoom_name_leaderID']= sessions_merged['zoom_name_leaderID'].str.lstrip('0')
sessions_merged['zoom_name_leaderID']= pd.to_numeric(sessions_merged['zoom_name_leaderID'],downcast='integer'). \
                                        fillna(0).astype('int')

sessions_merged=sessions_merged.rename(columns={ 'leaderID': 'email_leaderID'})

#Логика текущая такая: для каждого сеанса
#1. если есть в имени номер - используем его
#2. если в имени номера нет - берем leader_id
#Таким образом если с разными email но одним именем - то leaderID будет из имени браться и только при его отсутствии - из email

condlist = [sessions_merged.zoom_name_leaderID==0, sessions_merged.zoom_name_leaderID>0]
choicelist = [sessions_merged.email_leaderID, sessions_merged.zoom_name_leaderID]
sessions_merged['leaderID']= np.select(condlist,choicelist)

sessions_merged.head(1)

,user_name,meeting_id,email,time_on,День,event_type,untiID,email_leaderID,zoom_name_leaderID,leaderID
0,᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌ ᠌ ᠌᠌ ᠌ ᠌᠌ ᠌ ᠌ ᠌ ᠌,92998423778,bravo0987@bk.ru,146,2020-11-07,Акселерационная программа,311358.0,1578401,0,1578401


In [58]:
#sessions_merged.email.unique()
#pd.unique(sessions_merged.email[sessions_merged.email!=0]).tolist()
#Запишем  в файл 
#pd.unique(sessions_merged.email[sessions_merged.email!=0]).\
#to_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/email_list.csv', encoding='ANSI')  

k=sessions_merged[sessions_merged.email!=0].groupby('email').agg(leader_count=('leaderID', 'count')).reset_index()
k.to_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/email_list.csv', encoding='ANSI',columns=['email'],index=False) 

In [44]:
u=sessions_merged[sessions_merged.leaderID!=0].\
  groupby(['leaderID','participant_id']).agg(email_count=('email', 'count')).reset_index()
u.head()
#u.to_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_user_id.csv', encoding='ANSI',columns=['leaderID','participant_id'],index=False) 

,leaderID,participant_id,email_count
0,1,0,1
1,1,-4zsM_LKTGSFNV04Tjs66w,1
2,1,-G8Q7-GURs-TCq_y2I3iLg,2
3,1,0NAS9bXaQNOvoPmjxZeOeA,3
4,1,2A8cDQN_TQqxSCf_wE_qcA,2


In [63]:
#Теперь делаем преобразование
#1. Отбираем тех, кто присутствовал больше 10 минут -  time_on>600
#2. Группируем по: leaderID, event_type, День
#Считаем количество мероприятий каждого типа
#

session_by_type = sessions_merged[sessions_merged.time_on>600].\
    groupby(['leaderID','День','event_type']). \
    agg(zoom_count=('meeting_id', 'count')).fillna(0)
session_by_type=session_by_type.reset_index()

#Теперь все типы Зумов сделаем колонками
session_by_day = pd.pivot_table(session_by_type,index=['leaderID','День'],columns='event_type', values='zoom_count'). \
                reset_index().fillna(0)
#Удалим не типированные Зумы
session_by_day=session_by_day.drop(columns=[0])

session_by_day.tail(2)

event_type,leaderID,День,Акселерационная программа,Визионерская лекция,Лаборатория,Лекция,Мастер-класс,Общение с экспертами,Питч-сессии,Повышение эффективности
11252,16006847,2020-11-09,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11253,322238453,2020-11-07,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [64]:
# Сшиваем статистику с Зумами. Сшивка по полям leaderID и День
stat_merged = pd.merge(
    stat,
    session_by_day,
    on=['leaderID','День'],
    how='left' ).fillna(0)

#Переименуем в понятные всем имена - чтобы записи на мероприятия с Зумами не путались
stat_merged = stat_merged.rename(columns={ 'Акселерационная программа': 'Zoom по Акселерационной программе',
                         'Визионерская лекция':'Zoom по Визионерским лекциям',
                         'Лаборатория':'Zoom по Лабораториям',
                         'Лекция':'Zoom по Лекциям',
                         'Мастер-класс':'Zoom по Мастер-классам',
                         'Общение с экспертами':'Zoom по Общению с экспертами',
                         'Повышение эффективности': 'Zoom по Повышению эффективности'
                        })

stat_merged.head(3)

,leaderID,День,Загружена в PT презентация по проекту,Проект в ПТ размечен,Участником создана задача в Командном профиле,Выполнена задача в ком.профиле,Верифицировано решение в ком.профиле,Создание Темы изучения в Ком.профиле,Закрыта Тема изучения в Ком.профиле,Запись на Акслератор,...,Откликов на вакансии,untiID,Zoom по Акселерационной программе,Zoom по Визионерским лекциям,Zoom по Лабораториям,Zoom по Лекциям,Zoom по Мастер-классам,Zoom по Общению с экспертами,Питч-сессии,Zoom по Повышению эффективности
0,56,2020-11-07,0,0,0,0,0,0,0,0,...,0,24360,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
1,56,2020-11-08,0,0,0,0,0,0,0,0,...,0,24360,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,56,2020-11-09,0,0,0,0,0,0,0,0,...,0,24360,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [65]:
#Запишем  в файл 
stat_merged.to_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/engagement_v20_21_11_2020_fin.csv', \
                   encoding='ANSI', index=False)  